In [1]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point
import json

# Load the Layout Pool from Static Generation

In [2]:
with open("data/static_layout_pool.json", 'r') as openfile:
    json_object = json.load(openfile)

WIDTH = json_object["width"]
LENGTH = json_object["length"]
OUTPUT_FILE = "../Assets/Resources/json/dynamic.json"
layouts = [[Point(p[0], p[1]) for p in lay] for lay in json_object["layouts"]]
OBJECTS = json_object["objects"]

## Generation

- At least K = 5 trials between trials of the same pairing.
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    - Also, make sure these distances obey MIN_DIST_PATH

In [3]:
NUM_SHUFFLES = 50
DIST_BETWEEN_COPY = 5 # K
MIN_DIST_PATH = 1.5
LAYOUT_COUNT = len(layouts)

In [4]:
def generate_random_shuffle():
    # we interleave to random lists of 1...n/2, therefore it is guaranteed that the random shuffling
    #is valid for the constraint: boundary visibility alternates each time. each layout gets exactly 
    #one case with border, exactly one without border
    def random_half_shuffle():
        l = list(range(len(layouts)))
        random.shuffle(l)
        return l
    
    l = [x for t in zip(random_half_shuffle(), random_half_shuffle()) for x in t]
    return l

def is_valid_shuffle(order):
    # At least K = 5 trials between trials of the same pairing.
    # since we generate this as half and half, we only need to check the first and last K elements
    K = DIST_BETWEEN_COPY
    for i in range(K, len(order)):
        if order[i] in order[i-K:i]:
            #print("----", i, K, order[i], order[i-K:i])
            return False
    
    # each pair gets one occurence with border, one occurence without border
    # as a property of generate_random_shuffle, therefore we do not test for it.
    
    #TODO: We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    #TODO: Also, make sure these distances obey MIN_DIST_PATH
    
    rule_adjacentPairNotSameSide_violations = 0
    rule_minDistPath_violations = 0
    mirrored_y_for_rule = [False] # first layout is the anchor, never mirrored
    carrying_mirror_y = False
    
    for i in range(len(order) - 1):
        pre_point = layouts[order[i]][-1]
        post_point = layouts[order[i+1]][0]

        pre_side = pre_point.y < LENGTH/2
        post_side = post_point.y < LENGTH/2
        
        mirrored_pre_point = Point(pre_point.x, LENGTH - pre_point.y)
        mirrored_post_point = Point(post_point.x, LENGTH - post_point.y)
        
        # if there is any violation, try to mirror this next trial
        side_violation = pre_side == post_side
        dist_violation = (mirrored_pre_point if carrying_mirror_y else pre_point).dist(post_point) < MIN_DIST_PATH
        
        if (not side_violation) and dist_violation:
            return False # this will be impossible to solve, until, perhaps, x-mirroring is implemented
        
        if side_violation or dist_violation:
            # mirror across the border so that (hopefully) this violation won't happen
            carrying_mirror_y = not carrying_mirror_y
            side_violation = not side_violation # by definition, changing the mirroring causes a side violation
            
        mirrored_y_for_rule.append(carrying_mirror_y)
        
        # check rules again
        pre_for_checks = mirrored_pre_point if mirrored_y_for_rule[-2] else pre_point
        post_for_checks = mirrored_post_point if mirrored_y_for_rule[-1] else post_point
        
        
        dist_violation = pre_for_checks.dist(post_for_checks) < MIN_DIST_PATH
        
        if side_violation:
            return False
        if dist_violation:
            return False

    return rule_adjacentPairNotSameSide_violations + rule_minDistPath_violations == 0
        
def generate_valid_shuffle():
    shuffle = generate_random_shuffle()
    tries = 1
    while not is_valid_shuffle(shuffle):
        shuffle = generate_random_shuffle()
        tries += 1
    print(f"====== {tries} =====")
    return shuffle

In [5]:
shuffles = [generate_valid_shuffle() for i in range(3)]
print("\n".join([".".join([str(c) for c in shuf])[:10]  for shuf in shuffles]))

====== 274452 =====
====== 166170 =====
====== 446381 =====
48.41.20.2
11.26.28.2
10.45.46.4


In [17]:
generated_all = []

p2xy = lambda p: {"x": round(p.x, 3), "y": round(p.y, 3)}

for shuffle in shuffles:
    obj_count = 0
    generated_shuffle = []
    
    for shuffle_i in range(len(shuffle)):
        k = shuffle[shuffle_i]
        generated_objects = [p2xy(p) for p in layouts[k]]
        for obj_i in range(len(generated_objects) - 1):
            generated_objects[obj_i]["object"] = OBJECTS[obj_count]
            obj_count += 1
        generated_trial = {
            "targetObjects": generated_objects,
            "boundaryVisible": shuffle_i % 2
        }
        generated_shuffle.append(generated_trial)
    generated_all.append(generated_shuffle)
    
obj_count

500

In [18]:
with open(OUTPUT_FILE, "w") as outfile:
    outfile.write(json.dumps(generated_all))

## Checks

- We make sure that apmprox. half the pairs happen in A->B, and the other half happens in B->A
- We want to check the number of objects within the same side across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.